<a href="https://colab.research.google.com/github/SanchitaBora/Simple-Classification/blob/main/Differentiating_between_cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
from numpy.lib.npyio import zipfile_factory
!unzip train.zip
!unzip test1.zip

In [21]:
width, height = 224, 224

train = 'train'
test = 'test1' 
nb_train_sets = 12500
nb_test_sets = 12500
epochs = 10
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)


model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
  
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
  
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
  
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
  
test_datagen = ImageDataGenerator(rescale=1. / 255)
  
train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(width, height),
    batch_size=batch_size,
    class_mode='binary')
  
test_generator = test_datagen.flow_from_directory(
    test,
    target_size=(width, height),
    batch_size=batch_size,
    class_mode='binary')
  
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_sets // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_test_sets // batch_size)

model.save_weights('model_saved.h5')

model_json = model.to_json()
with open("./model.json","w") as json_file:
  json_file.write(model_json)




FileNotFoundError: ignored

In [ ]:
from keras.models import model_from_json
import cv2
import numpy as np

In [ ]:
json_file = open('./model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("./model.h5")
print("Loaded model from disk")

loaded_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

image = cv2.imread('./dataset_dogs_cats/test1.jpg')
image = cv2.resize(image, (50,50))
image = image.reshape(1, 50, 50, 3)

cv2.imshow("Input Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

result = loaded_model.predict_classes(image)
if(result[0][0] == 1):
    print("I guess this must be a Dog!")
else:
    print("I guess this must be a Cat!")